In [1]:
# Import Libraries and Tools
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Key Varibles

# Datasets
GeneralFolder = '/mnt/coronal_holes/Paper 2/Code 02 Observations/EUV_SingleChannel/'
stats1Folder = GeneralFolder + 'ScaledStats/'
stats8Folder = GeneralFolder + 'StandardStats/'
seeds1Folder = GeneralFolder + 'ScaledSeedStats/'
seeds8Folder = GeneralFolder + 'StandardSeedStats/'

# Carrington Rotation
CR = 'CR2133'

# Full Size Data Name
bigPrefix = 'ACWEresize_param1'

# Overwrite Previous Run
overwrite = False

# Dataset of Previous Run
seed1toseg1Utitle = 'Results/UniSeed1toseg1U.npz'
seed1toseg1Wtitle = 'Results/UniSeed1toseg1W.npz'
seed8toseg8Utitle = 'Results/UniSeed8toseg8U.npz'
seed8toseg8Wtitle = 'Results/UniSeed8toseg8W.npz'

# Figure Folder
figFolder = '/home/jgra/Coronal Holes/2nd paper/Code 02/Figures/SeedToSegEval/'

In [ ]:
# Get Data
stats1 = sorted(glob.glob(stats1Folder + CR + '/*/Unipolarity.*.npz'))
stats8 = sorted(glob.glob(stats8Folder + CR + '/*/Unipolarity.*.npz'))
seeds1 = sorted(glob.glob(seeds1Folder + CR + '/*/SeedUnipolarity.*.npz'))
seeds8 = sorted(glob.glob(seeds8Folder + CR + '/*/SeedUnipolarity.*.npz'))

In [ ]:
# Get Data

# Data from Prior Run
groupexits = os.path.exists(seed1toseg1Utitle) \
           & os.path.exists(seed1toseg1Wtitle) \
           & os.path.exists(seed8toseg8Utitle) \
           & os.path.exists(seed8toseg8Wtitle)

# New Run Procedure
if overwrite or not groupexits:
    
    # Prep
    seed1toseg1U = []
    seed1toseg1W = []
    seed8toseg8U = []
    seed8toseg8W = []
    
    # Cycle Through Cases
    for i in range(len(stats1)):

        # Get Specific Case
        stat1 = stats1[i]
        stat8 = stats8[i]
        seed1 = seeds1[i]
        seed8 = seeds8[i]

        #  Retreve Data - Skew1
        data = np.load(stat1, allow_pickle=True)
        lst  = data.files
        SEG1 = data[lst[0]]
        Skew1 = data[lst[1]]

        #  Retreve Data - Skew8
        data = np.load(stat8, allow_pickle=True)
        lst  = data.files
        SEG8 = data[lst[0]]
        Skew8 = data[lst[1]]

        #  Retreve Data - Seed1
        data  = np.load(seed1, allow_pickle=True)
        lst   = data.files
        SEED1 = data[lst[0]]
        SKW1S = data[lst[1]]

        #  Retreve Data - Seed8
        data  = np.load(seed8, allow_pickle=True)
        lst   = data.files
        SEED8 = data[lst[0]]
        SKW8S = data[lst[1]]

        #  Cycle through Seeds in Full Scale Image
        for j in range(len(SEED1)):

            # Get Seed
            seed1 = SEED1[j]

            #Get Skew - Seed
            compU = np.array([SKW1S[j,0],np.nan])
            compW = np.array([SKW1S[j,1],np.nan])

            # Find Best Segmentation
            for k in range(len(SEG1)):

                # Extract Region
                seg1 = SEG1[k]

                # Compare
                if np.sum((seg1 * seed1)) > 0:
                    compU[1] = Skew1[k,0]
                    compW[1] = Skew1[k,1]
                    break
                    
            # Save Results to List
            seed1toseg1U.append(compU)
            seed1toseg1W.append(compW)

        #  Cycle through Seeds in Eighth Scale Image
        for j in range(len(SEED8)):

            # Get Seed
            seed8 = SEED8[j]

            #Get Skew - Seed
            compU = np.array([SKW8S[j,0],np.nan])
            compW = np.array([SKW8S[j,1],np.nan])

            # Find Best Segmentation
            for k in range(len(SEG8)):

                # Extract Region
                seg8 = SEG8[k]

                # Compare
                if np.sum((seg8 * seed8)) > 0:
                    compU[1] = Skew8[k,0]
                    compW[1] = Skew8[k,1]
                    break
                    
            # Save Results to List
            seed8toseg8U.append(compU)
            seed8toseg8W.append(compW)

    # Convert to arrays
    seed1toseg1U = np.array(seed1toseg1U)
    seed1toseg1W = np.array(seed1toseg1W)
    seed8toseg8U = np.array(seed8toseg8U)
    seed8toseg8W = np.array(seed8toseg8W)
    
    # Save for Future Runs
    np.savez_compressed(seed1toseg1Utitle,seed1toseg1U)
    np.savez_compressed(seed1toseg1Wtitle,seed1toseg1W)
    np.savez_compressed(seed8toseg8Utitle,seed8toseg8U)
    np.savez_compressed(seed8toseg8Wtitle,seed8toseg8W)

# Load Existing Data
else:
    data  = np.load(seed1toseg1Utitle, allow_pickle=True)
    lst   = data.files
    seed1toseg1U = data[lst[0]]
    
    data  = np.load(seed1toseg1Wtitle, allow_pickle=True)
    lst   = data.files
    seed1toseg1W = data[lst[0]]
    
    data  = np.load(seed8toseg8Utitle, allow_pickle=True)
    lst   = data.files
    seed8toseg8U = data[lst[0]]
    
    data  = np.load(seed8toseg8Wtitle, allow_pickle=True)
    lst   = data.files
    seed8toseg8W = data[lst[0]]

In [ ]:
# Prepare for Scatter Plot
segUni = seed1toseg1U[:,1]
seedUni = seed1toseg1U[:,0]

# Plot - Full Scale
plt.figure(dpi=300)
plt.rcParams['axes.axisbelow'] = True
title = 'CR ' + CR[2:] + '\nUnweighted Unipolarity Relationship, Full Scale'
plt.title(title)
plt.scatter(seedUni,segUni,s=3,alpha=0.3,edgecolors='none',c='r')
plt.ylabel('Unipolarity of CH in EUV Only Segmentation')
plt.xlabel('Unipolarity of Initial Seed')
plt.grid()
plt.axis([-0.05,1.05,-0.05,1.05])
plt.savefig(figFolder+title.replace('\n',' ').replace(' ','_')+'.png')
plt.show()

# Provide Context
print('Total Number of Seeds:',len(seedUni))
print('Number of seeds ploted:',np.sum((~(np.isnan(seedUni))).astype(int)))
print()
print('Number of seeds with NaN unipolarity(not ploted):',np.sum(np.isnan(seedUni).astype(int)))
print('Number of final Segmentations not Found (NaN):',np.sum(np.isnan(segUni).astype(int)))

In [ ]:
# Prepare for Scatter Plot
segUni = seed1toseg1W[:,1]
seedUni = seed1toseg1W[:,0]

# Plot - Full Scale
plt.figure(dpi=300)
plt.rcParams['axes.axisbelow'] = True
title = 'CR ' + CR[2:] + '\nWeighted Unipolarity Relationship, Full Scale'
plt.title(title)
plt.scatter(seedUni,segUni,s=3,alpha=0.3,edgecolors='none',c='r')
plt.ylabel('Unipolarity of CH in EUV Only Segmentation')
plt.xlabel('Unipolarity of Initial Seed')
plt.grid()
plt.axis([-0.05,1.05,-0.05,1.05])
plt.savefig(figFolder+title.replace('\n',' ').replace(' ','_')+'.png')
plt.show()

# Provide Context
print('Total Number of Seeds:',len(seedUni))
print('Number of seeds ploted:',np.sum((~(np.isnan(seedUni))).astype(int)))
print()
print('Number of seeds with NaN unipolarity(not ploted):',np.sum(np.isnan(seedUni).astype(int)))
print('Number of final Segmentations not Found (NaN):',np.sum(np.isnan(segUni).astype(int)))

In [ ]:
# Prepare for Scatter Plot
segUni = seed8toseg8U[:,1]
seedUni = seed8toseg8U[:,0]

# Plot - 1/8th Scale
plt.figure(dpi=300)
plt.rcParams['axes.axisbelow'] = True
title = 'CR ' + CR[2:] + '\nUnweighted Unipolarity Relationship, Eighth Scale'
plt.title(title)
plt.scatter(seedUni,segUni,s=3,alpha=0.3,edgecolors='none',c='r')
plt.ylabel('Unipolarity of CH in EUV Only Segmentation')
plt.xlabel('Unipolarity of Initial Seed')
plt.grid()
plt.axis([-0.05,1.05,-0.05,1.05])
plt.savefig(figFolder+title.replace('\n',' ').replace(' ','_')+'.png')
plt.show()

# Provide Context
print('Total Number of Seeds:',len(seedUni))
print('Number of seeds ploted:',np.sum((~(np.isnan(seedUni))).astype(int)))
print()
print('Number of seeds with NaN unipolarity(not ploted):',np.sum(np.isnan(seedUni).astype(int)))
print('Number of final Segmentations not Found (NaN):',np.sum(np.isnan(segUni).astype(int)))

In [ ]:
# Prepare for Scatter Plot
segUni = seed8toseg8W[:,1]
seedUni = seed8toseg8W[:,0]

# Plot - 1/8th Scale
plt.figure(dpi=300)
plt.rcParams['axes.axisbelow'] = True
title = 'CR ' + CR[2:] + '\nWeighted Unipolarity Relationship, Eighth Scale'
plt.title(title)
plt.scatter(seedUni,segUni,s=3,alpha=0.3,edgecolors='none',c='r')
plt.ylabel('Unipolarity of CH in EUV Only Segmentation')
plt.xlabel('Unipolarity of Initial Seed')
plt.grid()
plt.axis([-0.05,1.05,-0.05,1.05])
plt.savefig(figFolder+title.replace('\n',' ').replace(' ','_')+'.png')
plt.show()

# Provide Context
print('Total Number of Seeds:',len(seedUni))
print('Number of seeds ploted:',np.sum((~(np.isnan(seedUni))).astype(int)))
print()
print('Number of seeds with NaN unipolarity(not ploted):',np.sum(np.isnan(seedUni).astype(int)))
print('Number of final Segmentations not Found (NaN):',np.sum(np.isnan(segUni).astype(int)))